In [1]:
import os
import pandas as pd
import xgboost
import utils
import scoring
from sklearn.model_selection import train_test_split

In [2]:
# The datasets are available in CoCalc in ~/share/data/I-coopetition-muon-id/
# Test
# ! wget --content-disposition https://codalab.coresearch.club/my/datasets/download/dd6255a1-a14b-4276-9a2b-db7f360e01c7
# Train
# ! wget --content-disposition https://codalab.coresearch.club/my/datasets/download/3a5e940c-2382-4716-9ff7-8fbc269b98ac

In [3]:
columns = utils.SIMPLE_FEATURE_COLUMNS + ["id", "label", "weight", "sWeight", "kinWeight"]
# DATA_PATH = "."
DATA_PATH = "~/share/data/I-coopetition-muon-id/"
train = pd.read_csv(os.path.join(DATA_PATH, "train.csv.gz"), index_col="id", usecols=columns)
test = pd.read_csv(os.path.join(DATA_PATH, "test-features.csv.gz"), index_col="id", usecols=utils.SIMPLE_FEATURE_COLUMNS + ["id"])

/home/nkazeev/.conda/envs/py3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
train.head()

,ncl[0],ncl[1],ncl[2],ncl[3],avg_cs[0],avg_cs[1],avg_cs[2],avg_cs[3],ndof,MatchedHit_TYPE[0],...,Mextra_DY2[1],Mextra_DY2[2],Mextra_DY2[3],FOI_hits_N,PT,P,sWeight,label,kinWeight,weight
id,,,,,,,,,,,,,,,,,,,,,
0,79,18,22,10,2.392405,2.055556,1.590909,1.200000,8,2,...,5501.422000,12634.98400,25386.30500,8,1230.391860,5488.168681,1.190017,1,3.00000,3.570052
1,33,14,6,15,1.575758,2.285714,1.333333,1.400000,8,2,...,671.103200,1497.54480,2901.06470,4,2360.763287,15284.170042,-0.414881,1,1.85646,-0.770209
2,115,53,23,25,2.843478,2.433962,1.826087,1.280000,8,2,...,917.825600,2065.64230,4027.29760,4,2685.870252,13461.339980,-0.390778,1,3.00000,-1.172334
3,28,8,5,15,2.964286,1.500000,1.800000,1.000000,8,2,...,25.208195,55.87817,106.25816,7,6015.405116,74550.106103,1.218314,1,1.85646,2.261752
4,50,8,4,13,2.400000,1.250000,1.500000,1.076923,8,2,...,408.524720,917.87640,1773.50920,5,1507.013894,19206.813287,-0.387133,1,1.85646,-0.718697


In [5]:
test.head()

,ncl[0],ncl[1],ncl[2],ncl[3],avg_cs[0],avg_cs[1],avg_cs[2],avg_cs[3],ndof,MatchedHit_TYPE[0],...,Mextra_DX2[1],Mextra_DX2[2],Mextra_DX2[3],Mextra_DY2[0],Mextra_DY2[1],Mextra_DY2[2],Mextra_DY2[3],FOI_hits_N,PT,P
id,,,,,,,,,,,,,,,,,,,,,
0,55,12,10,15,2.127273,1.166667,1.500000,1.600000,8,2,...,3431.287800,7550.342300,14739.10600,1210.756600,3462.087200,7618.11100,14871.39550,6,1174.484530,6884.502272
1,43,18,12,15,2.697675,1.111111,1.333333,1.600000,8,2,...,63.777058,140.748630,266.41522,21.983400,63.552402,140.25285,265.47680,7,4344.639775,48769.526891
2,90,34,19,29,1.800000,2.058824,1.210526,1.344828,8,2,...,401.787050,903.642400,1749.45590,133.357620,383.406950,862.30440,1669.42520,4,4109.805750,20296.048412
3,141,43,15,16,4.609929,2.744186,1.466667,1.125000,8,2,...,211.605320,471.151430,900.27890,72.982890,211.607040,471.15527,900.28613,9,940.242439,26062.046122
4,22,18,8,9,1.636364,2.444444,1.000000,1.555556,8,2,...,39.059680,85.982086,163.04549,13.728162,39.018433,85.89129,162.87330,8,1386.377851,59872.343332


In [6]:
train_part, validation = train_test_split(train, test_size=0.25, shuffle=True, random_state=2342234)

In [7]:
model = xgboost.XGBClassifier(n_jobs=-1)

In [8]:
model.fit(train_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values,
          train_part.label.values,
          sample_weight=train_part.kinWeight.values)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [9]:
validation_predictions = model.predict_proba(validation.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]

In [10]:
scoring.rejection90(validation.label.values, validation_predictions, sample_weight=validation.weight.values)

0.5775103906760958

In [11]:
model.fit(train.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values, train.label, sample_weight=train.kinWeight.values)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [12]:
predictions = model.predict_proba(test.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]

In [16]:
compression_opts = dict(method='zip',
                        archive_name='submission.csv')  
pd.DataFrame(data={"prediction": predictions}, index=test.index).to_csv(
    "submission.zip", index_label=utils.ID_COLUMN, compression=compression_opts)